In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Variational Autoencoder (VAE)

The "Vanilla" Autoeconder that we studied
- the Encoder maps example $\x^\ip$
- to a single latent representations $\z^\ip$
- such that the Decoder tries to "invert" $\z^\ip$ to recover an approximation of $\x^\ip$

The hope was that a slight perturbation in $\z^\ip$ 
- would invert into an $\x_\text{synthetic}^\ip$
- that was a member of the valid class of examples

thus facilitating the use of the Decoder as a source of synthetic examples.

However, this is what we get
- if we invert latent represenations
- that are near neighbors of $\z^\ip$

As you see
- even small changes to $\z^\ip$ result in corrupted (i.e., not members of the 10 classes of clothing items) examples

<img src="images/autoencoder_perturb_single_img.png" width=90%>



This motivates the *Variational Autoencoder (VAE)* 
- an Encoder that maps example $\x^\ip$ to a *distribution* of latents
- such that samples from the distribution
- can be inverted by the Decoder
- into reasonable approximations of $\x^\ip$

# VAE: An engineering perspective 

The Encoder part of a VAE, given input $\x^\ip$
- Produces the parameters (e.g., $\mu^\ip, \sigma^\ip$) of a distributional form
- Draws a sample from the distribution as its output $\z^\ip$

Thus, the latent representation $\z$ of a given $\x$ is a probability distribution $\qrs{\z \mid \x}{\Phi}$


<table>
    <tr>
        <th><center>Variational Autoencoder (VAE)</center></th>
    </tr>
    <tr>
        <td><img src="images/Autoencoder_VAE.png"></td>
    </tr>
</table>

**Note**

$\mu^\ip$ and $\sigma^\ip$ are 
- vectors
- computed values (and hence, functions of $\x^\ip$) and **not** parameters
- so training learns a *function* from $\x^\ip$ to $\mu^\ip$ and $\sigma^\ip$


Although the Encoder 
- creates a distribution of latents $\qrs{\z \mid \x}{\Phi}$

it will be necessary to constrain this distribution
- to be an approximation of the "true" distribution $\qr{\z \mid \x}$ of latents

Thus, when training the VAE, the Loss Function will have two terms
- the Reconstruction Loss
    - enforcing that the reconstruction  $\tilde{x}^\ip$ is close to the input $\x^\ip$
- the Divergence Constraint
    - ensuring that that constructed distribution $\qrs{\z | \x}{\Phi}$ is close to true distribution $\qr{\z | \x}$
$$
\KL( \qrs{\z | \x}{\Phi} \; ||\; \qr{\z | \x})
$$

This is not just straightforward engineering.

In fact: the architecture of the VAE was obtained from the math rather than vice-versa !

We provide a brief overview of the mathematics.

The interested reader is referred to a highly recommended [VAE tutorial](https://arxiv.org/pdf/1606.05908.pdf)
for a detailed presentation.


# Details

**Notation summary**

term &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| dimension &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | meaning 
:---|:---|:---
$\x$ | $n$ | Random variable for Input
$\tilde\x$ | $n$ | Output: reconstructed $\x$
$\z$ | $n' << n$ | Random variable for Latent representation
$E$  | $\mathbb{R}^n \rightarrow \mathbb{R}^{n'}$ | Encoder
| | $E(\x) = \z $
$D$  | $\mathbb{R}^{n'} \rightarrow \mathbb{R}^n$ | Decoder
| | $\tilde\x = D(\z) $
| | $\tilde\x = D( E(\x) )$
| | $\tilde\x \approx \x$
$\pr{\x}$ | prob. distribution | *prior* distribution of Inputs
          | | intractable.  Only have empirical.
$\qr{\z}$ | prob. distribution | *prior* distribution of Latents
$\qr{\z \mid \x}$ | prob. distribution| *posterior* marginal distribution of Latents given Input
                  | | intractable
$\qrs{\z \mid \x}{\Phi}$ | Neural Network| NN to approximate $\qr{\z \mid \x}$ 
                         | | Encoder
$\prs{\x \mid \z}{\Theta}$ | Neural Network| NN to approximate $\pr{\x \mid \z}$ 
                  | | Decoder

# VAE: Mathematical perspective

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>The VAE has a very interesting <b>two part</b> Loss Function</li>
        <ul>
            <li>Reconstruction Loss, as in the Vanilla AE</li>
            <li>Divergence Loss
        </ul>
        <li>The Reconstruction Loss is not sufficient</li>
        <ul>
            <li>Issues of intractability arise</li>
            <li>The Divergence Loss skirts intractability</li>
            <ul>
                <li>By constraining the Encoder to produce a tractable distribution</li>
            </ul>
        </ul>
    </ul>
</div>

## Distributions of inputs, latents, outputs

Let's pretend that we don't already know the architecture of a VAE
- that the latent $\z^\ip$ is generated as a probability function of $\mu^\ip$ and $\sigma^\ip$ given input $\x^\ip$.

Instead let
- $\x \sim \pr{\x}$ denote
    - an example $\x$ drawn from the distribution of examples $\pr{\x}$
- $\z \sim    \qr{\z}$ denote
    - a latent drawn from the true distribution of latents $\qr{\z}$

We marginalize these distributions to 
- show the dependence between example $\x^\ip$ and its latent representations

Let
- $\qr{\z | \x}$ denote the *conditional*  distribution of Latent, given an Input
    - This is the distribution that is produced by the Encoder
- $\pr{\x | \z}$: the *conditional* distribution of Input, given a Latent
    - This is the distribution that is produced by the Decoder

We can go from the marginal back to the joint distribution by integration
- $\pr{\x | \z}$
- over all $\z \in \qr{\z}$

$$
\pr{\x} = \int_{\z \in \qr{\z}}{ \pr{\x | \z} \; \qr{\z} }
$$



Operationally, this would result in a Decoder architecture

<table>
    <tr>
        <th><center>VAE derivation: 1</center></th>
    </tr>
    <tr>
        <td><img src="images/Autoencoder_VAE_derivation_B_0.png" width=80%></td>
    </tr>
</table>

Some obvious concerns about the integral
- It may be very expensive to draw many samples of $\z$ from $\qr{\z}$ for each training example $\x^\ip$
- There are many random choices of $\z$ from $\qr{\z}$ which can't reconstruct $\x^\ip$
    - i.e., $\prs{\x^\ip | \z'}{\Theta} = 0$ for many $\z'$


We can improve our sampling by considering only those choices of $\z$ that could generate $\x$
and re-write the objective as

$$
\pr{\x} = \int_{ \z \in \qr{\z | \x} } { \pr{\x | \z} \; \qr{\z} }
$$


## Problem: Some distributions aren't known !

But there's a problem !
- The distribution $\pr{\x}$ of inputs $\x$ is *intractable*
- We only have *empirical samples* from the distribution
    - i.e., the training examples
    - we don't know the functional form of the parent distribution
    
For instance: what is the distribution of images of Dogs ?

Similarly
- $\qr{\z | \x}$ is intractable
    - Recall: this is the intended output of the Encoder

## Operationalizing the mathematically-derived architecture

We will side-step the intractability issues by defining Neural Networks to learn
*approximations* 
- $\qrs{\z | \x}{\Phi}$
    - a Neural Network (the Encoder)
    - with parameters $\Phi$
    - that will use training examples
- to learn an approximation of true $\qr{\z | \x}$
    

Similarly
- $\prs{\x | \z}{\Theta}$
    - a Neural Network (the Decoder)
    - with parameters $\Theta$
    - that will use training examples
- to learn an approximation of true $\pr{\x | \z}$



The resulting architecture:


<table>
    <tr>
        <th><center>VAE derivation: 2</center></th>
    </tr>
    <tr>
        <td><img src="images/Autoencoder_VAE_derivation_B.png"></td>
    </tr>
</table>

## Constraining the Encoder

In order to ensure that the Encoder's
- approximate distribution $\qrs{\z | \x}{\Phi}$
- is close to true distribution $\qr{\z | \x}$

we add the Divergence Constraint to the Loss Function

$$
\KL( \qrs{\z | \x}{\Phi} \; ||\; \qr{\z | \x})
$$


**Note**

The KL function is *not symmetric*
$$
\KL( \qrs{\z | \x}{\Phi} \; ||\; \qr{\z | \x}) \ne \KL(  \qr{\z | \x} ||\; \qrs{\z | \x}{\Phi} )
$$

So the constraint, as written
- says that each approximation 
- is close to the true value

It *does not* say
- that each true value
- has a close approximation

## The Loss Function for training

We can train the Encoder/Decoder pair with the objective that the
reconstructed $\tilde{\x}^\ip$ approximates the true $\x^\ip$ from the training set, across all examples $i$.

One way of stating this is as a Maximum Likelihood:
- Solve for the weights $\Phi, \Theta$
- That maximize the (log) Likelihood of the set of reconstructions $\tilde{\X}$ reproducing the training set $\X$

Since our practice is to minimize Loss (rather than maximize an objective function)
we write our loss as (negative of log) likelihood
$$
\begin{array}[llll] \\
\loss & = & - \log( \pr{\X} )
\end{array}
$$

Minimizing $\loss$ is equivalent to maximizing likelihood.



Adding the KL divergence constraint to our Likelihood objective gives the loss function
$$
\begin{array}[lll]\\
\loss  & = & - \log(\prs{\x}{\Theta}) + \KL( \qrs{\z|\x}{\Phi} \; ||\; \qr{\z | \x}) \\
& = & \loss_R + \loss_D
\end{array}
$$

which now has two objectives
- Reconstruction loss $\loss_R$: maximize the likelihood (by minimizing the negative likelihood)
- Divergence constraint $\loss_D$: $\qrs{\z|\x}{\Phi}$ must be close to $\qr{\z | \x}$

$$
\begin{array}[llll] \\
\loss_R & = & - \log( p_\theta(\x ) ) \\
\loss_D & = & \KL \left(  \qrs{\z|\x}{\Phi} \; || \; \qr{\z | \x} \right) \\
\end{array}
$$

## Problem: $\loss$ contains an intractable term

Looking closely as the Divergence Constraint
$$
\loss_D  =  \KL \left(  \qrs{\z|\x}{\Phi} \; || \; \qr{\z | \x} \right) 
$$

we note the presence of the term
$$
\qr{\z | \x}
$$

which we identified as intractable.

Hence
- the Divergence Constraint
- as written
- cannot be operationalized

We will show (in the next section: lots of algebra !) that the loss can be re-written as
$$
\loss = - \E_{\z \sim \qrs{\z|\x}{\Phi}}\left( \log( \prs{\x|\z}{\Theta} ) \right) + \KL(\qrs{\z|\x}{\Phi}  \; ||\;  \qr{\z} )
$$

This is *almost* identical to our original express for $\loss$ except
- the intractable KL term
$$
\KL \left(  \qrs{\z|\x}{\Phi} \; || \; \qr{\z | \x} \right)
$$

becomes 
$$
 \KL \left(  \qrs{\z|\x}{\Phi} \; || \; \qr{\z} \right)
$$

This is still contains intractable
$$
\qr{\z}
$$

We can make this tractable
- by making *an assumption* as to the functional form of $\qr{\z}$
    - e.g., Normal
    

**Note**

it is easier to make an assumption about 
- *unconditional* distribution $\qr{\z}$

rather than
- *conditional* distribution $\qr{\z | \x}$

since we don't have to know the joint relationship between example $\x$ and its latent $\z$

## Choosing $\qr{\z}$

So what distribution should we use for the prior $\qr{\z}$ ?
- It should be differentiable, since we use Gradient Descent for optimization
- It should be tractable with a closed form (such as a normal)

A *convenient* (but **not necessary**) choice for $\qr{\z}$ is normal
- If we choose $\qr{\z}$ as normal, we can require $q_\phi( \z | \x )$ to be normal too
- The KL divergence between two normals is an easy to compute function of their means and standard deviations.
    - the "easy to compute" part is the "convenience"
    - See [VAE tutorial](https://arxiv.org/pdf/1606.05908.pdf) Section 2.2


## Re-parameterization trick

There is still one impediment to training.

It involves the random choice of $\z \sim \qrs{\z|\x}{\Phi}$ in

$$
\loss_R = \E_{z \sim \qrs{\z|\x}{\Phi} } \left( \log( \prs{\x | \z}{\Theta} ) \right)
$$

This is not a problem in the forward pass.

But in the backward pass we need to compute
$$
\frac{\loss_R}{\partial \Theta}
$$

How do we back propagte through a random choice ?


The "reparameterization trick" redefines the random choice $\z$ as

$$
\begin{array}[llll] \\
\mathbf{z}  & = & \mathbf{\mu}_\theta(\x) + \mathbf{\sigma}_\theta(\x) * \mathbf{\epsilon} \\
\mathbf{\epsilon} & \sim & N(0,1) \\
\end{array}
$$

That is
- Once we have defined $\qr{\z}$ to be a Normal distribution
- We can re-write an element of the distribution
    - as the distribution's mean plus a random standardized Normal $\epsilon$ times the distribution's standard deviation
    
In this formulation, the random variable $\epsilon$ appears in a product term
- We can differentiate the product with respect to $\Theta$
- $\epsilon$ can be treated as a constant in $\frac{\partial \epsilon}{\partial \Theta}$

The Encoder design is now to produce
(trainable parameters) $\mu_\Theta, \sigma_\Theta$
- And $\z$ indirectly


<table>
    <tr>
        <th><center>Reparameterization trick</center></th>
    </tr>
    <tr>
        <td><img src="images/Reparameterization_trick.png"></td>
    </tr>
</table>

This gets us to the  final picture of the VAE:

<table>
    <tr>
        <th><center>Variational Autoencoder (VAE)</center></th>
    </tr>
    <tr>
        <td><img src="images/Autoencoder_VAE.png"></td>
    </tr>
</table>

To train a VAE:
- pass input $\x^\ip$ through the Encoder, producing $\mu^\ip, \sigma^\ip$
    - use $\mu^\ip, \sigma^\ip$ to sample a latent representation $\z^\ip$ from the distribution
- pass the sampled $\z^\ip$ through the decoder, producing $D(\z^\ip)$
- measure the reconstruction error $\x^\ip - D(\z^\ip)$, just as in a plain AE
- back propagate the error, updating all weights and $\mu, \sigma$

Each time that we encounter the same training example (e.g., in different epochs), we select another random element from the distribution.

Thus the VAE learns to represent the same example from multiple latents.


# Using a VAE to produce synthetic examples



We can examine how the latent representations produced by the VAE form clusters:

<table>
   <tr>
       <th><center>MNIST examples: clustering of latent $\z$</th>
    </tr>
    <tr>
        <td><img src="images/vae_latents.png" width=80%></td>
    </tr>

</table>

In comparing the clusterings produced by the VAE against our previous example of a plain Autoencoder be aware
- The two models are displaying results on different data: MNIST digits verus Fashion MNIST
- The architecture of the Encoder and Decoder are different in the two models
    - The plain Autoencoder used extrememly simple architectures
        - Could the more complex architecture of the VAE Encoder/Decoder be the cause of tighter clustering ?
        
Certainly room for experimentation !

## Can we interpret the elements of the latent vector ?

The latent vector can be thought of as a
- reduced dimensionality
- representation of $\x^\ip$

where each element of $\z^\ip$ is in
- a new basis space
- of "concepts"
- rather than syntactic features

This is similar to the ideas we expressed 
- when trying to interpret Principal Components

We will attempt an interpretation by  
- examining *the neighborhood* around the latent $\z^\ip$ of example $\x^\ip$
- by perturbing each element of the latent vector

In our illustration
- latent vectors $\z$ that are implemented as a vectors of length $2$
    - as are $\mu$ and $\sigma$

We vary the first and second element of a vector 

- and obtain the Decoder output on each combination of first and second element
   

So the results are *possible* outputs of the Decoder
- based on latent vectors $\z$
- that are not necessarily the output of the Encoder on any actual input $\x$

<table>
   <tr>
       <th><center>Synthetic MNIST examples from a VAE:<br> Code implements latent $\z$ as vector with length 2 <br> vary the 2 elements of the latent vector </th>
    </tr>
    <tr>
        <td><img src="images/vae_outputgrid.png" width=90%></td>
    </tr>

</table>

Can we interpret $\z_0$ and $\z_1$ by inspecting how the Decoder output changes
by varying each ?

- $\z_0$: controls slant ?
    - See the bottom row of $0$'s
- $\z_1$: controls "verticality" ?
    - See right-most column

# Conditional VAE

The VAE would seem to offer a solution to the problem of creating synthetic data.

But there is a problem
- an *unlabeled* example is created
- we have no way of knowing the label
- nor do we have a way of *controlling* the output so as to be an example with a specified label

We can modify the VAE so as to *conditionally* generate an example with a specified label.
- [Conditional VAE](Cond_VAE_Generative.ipynb)

# Advanced details

## Obtain $\loss$ by rewriting $\KL( \qrs{\z|\x}{\Phi} \; ||\; \qr{\z|\x} )$

Let's derive a simpler expression for $\loss$ by manipulating $\KL( \qrs{\z|\x}{\Phi} \; ||\; \qr{\z|\x})$:

$
\begin{array}[llll]\\
\KL( \qrs{\z|\x}{\Phi} \; ||\; \qr{\z | \x}) &  = & \sum_z{ \qrs{\z|\x}{\Phi}(\log(\qrs{\z|\x}{\Phi}) - \log(\qr{\z | \x}) } & \text{def. of KL} \\
&  = & \E_{z \sim \qrs{\z|\x}{\Phi} } \left( \log(\qrs{\z|\x}{\Phi}) - \log(\qr{\z | \x}) \right) & \text{def. of }\E \\
&  = & \E_{z \sim \qrs{\z|\x}{\Phi}) } ( \; \log(\qrs{\z|\x}{\Phi}) \\ & & -\left( \; \log( \pr{\x | \z}) + \log(\qr{\z}) - \log(\pr{\x} \right)    \,   )  \;\;)&  \text{Bayes theorem on } \\
 & & & \log(\qr{\z|\x}) \\
\KL( \qrs{\z|\x}{\Phi} \; ||\; \qr{\z | \x}) \\ - \log(\pr{\x}) & = & \E_{z \sim \qrs{\z|\x}{\Phi} } \left( \; \log(\qrs{\z|\x}{\Phi})  - \left( \log( \pr{\x | \z} ) + \log( \qr{\z} ) \right) \;\right) & \text{ move } \log(\pr{\x}) \text{ to LHS} \\
 & = & \E_{z \sim \qrs{\z|\x}{\Phi} } \left( \; - \log( \pr{\x | \z} ) + ( \; \log(\qrs{\z|\x}{\Phi})  - \log( \qr{\z} ) \; )     \; \right) & \text{re-arrange terms} \\
 & = & - \E_{z \sim \qrs{\z|\x}{\Phi} } \left( \log( \pr{\x | \z} ) \right) + \KL(\qrs{\z|\x}{\Phi} \; ||\;  \qr{\z} ) & \text{def. of KL} \\
 \loss & = & - \E_{z \sim \qrs{\z|\x}{\Phi} } \left( \log( \pr{\x | \z} ) \right) + \KL(\qrs{\z|\x}{\Phi} \; ||\;  \qr{\z} ) & \text{since LHS} = \loss \\
\end{array}
$

**The key step**:
- Using Bayes Theorem to re-write
$$\log(\qr{\z|\x}) $$
as
$$
\log( \pr{\x | \z}) + \log(\qr{\z}) - \log(\pr{\x} )
$$
- This allows us do away with intractable conditional probability $\qr{\z|\x}$
- In favor of unconditional probability $\qr{\z}$

The LHS cannot be optimized via SGD (recall the tractability issue with  $\qr{\z|\x}$).

**But the RHS can be made tractable** by
- Approximating $\pr{\x | \z}$ with $\prs{\x | \z}{\Theta}$
- Assuming that the distributions $\qr{\z}$ (and $\pr{\x}$) as Normal




## Code 
 
Here is a diagram of the code for the Encoder side of the VAE.

Note that the sub-networks that compute
- $\mu$ (box labeled `z_mean`)
- $\sigma$ (box labeled `z_log_var`)

have outputs of length $2$ so  latent vector $\z$ has two elements.

We will use latents of length $2$ in an example below showing
some possible outputs of the Decoder.


Notice that
- the inputs to the sub-network computing $\mu$ and $\sigma$
- are identical

<table>
    <th><center>VAE: Components (Encoder)</center></th>
    <tr>
        <td><center><strong>Encoder</bold></strpmg></td>
    </tr>
    <tr>
        <td><img src="images/vae_conv_encoder.png" width=90%></td>
    </tr>

</table>

### VAE code highlights

[Here](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb#scrollTo=HPRXyb06O2y8) is an implementation of the VAE.

Highlights
- `encoder` samples from the distribution

        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    

- custom `train_step`
    - computes two-part loss: Reconstruction Loss + KL Loss

        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)
                reconstruction = self.decoder(z)
                reconstruction_loss = tf.reduce_mean(
                    tf.reduce_sum(
                        keras.losses.binary_crossentropy(data, reconstruction),
                        axis=(1, 2),
                    )
                )
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
                total_loss = reconstruction_loss + kl_loss

## Code: VAE, CVAE

We can learn much more about the properties and use of a VAE through examples

A secondary objective is to look at the code which involves some advanced features of Keras.

The architecture of the VAE will be more complex compared to the vanilla Autoencoder.

<table>
   <tr>
            <th>VAE: Components</th>
    </tr>
    <tr>
        <td><center><strong>Encoder</bold></strpmg></td>
        <td><center><strong>Decoder</bold></strpmg></td>
    <tr>
        <td><img src="images/vae_conv_encoder.png" width=90%></td>
        <td><img src="images/vae_conv_decoder.png" width=90%></td>
    </tr>

</table>

Encoder
- Note the two branches to nodes `z_mean` and `z_log_var`
    - The output of their common parent is used to generate two separate values ($\mu$ and $\sigma$)
    - $\mu$ and $\sigma$ are both vectors of length $2$
        - Thus, the sampled $\z$ is also of length $2$
        - In our grid illustration of generating synthetic examples, we vary each of the $2$ components of $\z$
    - Latent is *much* shorter than in the plain VAE
        - does the random nature of sampling facilitate shorter representation ?

Let's explore the code through this [notebook](VAE_code.ipynb)
- VAE code
- CVAE code

## Loss function: discussion

Let's examine the role of $\loss_R$ and $\loss_D$ in the loss function $\loss$.

- What would happen if we dropped $\loss_D$ ?
    - We would wind up with a deterministic $\z$ and collapse to a vanilla VAE
    
- What would happen if we dropped $\loss_R$ ?
    - the encoding approximation $\qrs{\z|\x}{\Phi}$ would be close to the empirical $\pr{\z | \x}$ *in distribution*
    - but two variables with the same distribution are not necessarily the same ?
        - e.g., get a distribution $p$ by flipping a coin
            - let distribution $q$ be a relabelling of $p$ by changing Heads to Tails and vice-versa
            - $p$ and $q$ are equal in distribution but clearly different !
    


## ELBo (Evidence-based Lower Bound)

By re-writing the Loss, we removed the intractable term $\qr{\z|\x}$

It turns out that even this may not be necessary.

For the truly interested reader:
- The derivation uses a method known as *Variational Inference*.  See this 
[blog](https://mbernste.github.io/posts/variational_inference/) for a summary.
- One can show that loss $\loss$ is equal to $-1$ times the *ELBo* (Evidence Based Lower Bound)

So if one knows how to maximize the [ELBo](https://mbernste.github.io/posts/elbo/), one can minimize the loss.
     

In [2]:
print("Done")

Done
